In [11]:
import numpy as np
import pandas as pd
from scipy.stats import laplace
import matplotlib.pyplot as plt
import math
import gc
import inspect
gc.enable()

### **Lendo dados**

In [12]:
adult = pd.read_csv("./data/raw/adult.csv")
adult = adult.rename(columns = {"class":"Classe"})

### ***Definindo consultas***

In [13]:
def Consulta(q, data, colname):
    Q = {'avg': np.mean,
         'soma': np.sum}
    return(Q[q](data[colname])) 

In [14]:
def Contagem(dataset, condicao):
    return(dataset[(dataset.Classe == condicao)].count()[1])

In [15]:
def agg(dataset):
    Consultas = [Consulta('avg', dataset, 'age'),
                 Consulta('soma', dataset, 'capitalgain'),
                 Consulta('avg', dataset, 'hoursperweek'),
                 Contagem(dataset, ">50K")
                ]
    return(Consultas)

### ***Calculo sensibilidade Global***

In [16]:
def distancia(bd1, bd2):
    dists = [0]*4
    a, b = agg(bd1), agg(bd2)
    for i in range(0, 3):
        dists[i] = np.linalg.norm(a[i]-b[i])
    return(dists)

In [17]:
Deltas = pd.DataFrame({ 'q1':[], 'q2':[], 'q3':[], 'q4':[] })

In [18]:
for index, row in adult.iterrows():
    Distancia = pd.DataFrame(distancia(adult, adult.drop(index)))
    Distancia.index, Distancia = ['q1', 'q2', 'q3', 'q4'], Distancia.transpose()
    Deltas = Deltas.append(Distancia)  
gc.collect()

14

In [19]:
MaxDeltas = Deltas.apply(lambda x: max(x))
MaxDeltas[3] = 1
Q=['Avg(age)', "Sum(capitalgain)", 'Avg(hoursperweek)', 'Count(>50K)']
for i in range(0, 4):
    print("\nSensibilidade", Q[i]+":", '\n', MaxDeltas[i], "\t")


Sensibilidade Avg(age): 
 4.5636285693984036e-05 	

Sensibilidade Sum(capitalgain): 
 4.0 	

Sensibilidade Avg(hoursperweek): 
 4.19586378328507e-05 	

Sensibilidade Count(>50K): 
 1.0 	


#### **Adicionando ruido à consulta**

In [ ]:
datasetFinal = pd.DataFrame(datasetFinal, 
                            columns = ['budget', 
                                       'q1',
                                       'q2', 
                                       'q3', 
                                       'q4', 
                                       'sens q1',
                                       'sens q2',
                                       'sens q3',
                                       'sens q4'])

In [21]:
eps = [0.1, 1, 10]
def Priv(data, sens, eps):
    err = []
    sens = np.asarray(sens)
    for j in range(1, len(eps)+1):
        for i in range(1, len(sens)+1):
            err.append((laplace.rvs((sens[i-1]/eps[j-1]), 0)))
    err = np.array(err)
    q = np.repeat(np.array(agg(adult)), [3 for i in range(0, len(sens))], axis = 0)
    results = pd.DataFrame({
        'x~Laplace':err, 
        'Sens.':np.repeat(sens, [3 for i in range(0, len(sens))], axis = 0),
        'Q+x':q+err,
        'Q':q
    }).transpose()
    return(results)

In [23]:
MaxDeltas

q1    0.000046
q2    4.000000
q3    0.000042
q4    1.000000
dtype: float64

In [22]:
sens = [4.5636285693984036e-05, 4, 4.19586378328507e-05, 1]
Priv(adult, sens, eps)

,0,1,2,3,4,5,6,7,8,9,10,11
x~Laplace,0.000456,40.000000,0.000420,10.0,0.000046,4.0,0.000042,1.000000,0.000005,0.4,0.000004,0.1
Sens.,0.000046,0.000046,0.000046,4.0,4.000000,4.0,0.000042,0.000042,0.000042,1.0,1.000000,1.0
Q+x,1.771535,41.771078,1.771498,9794.0,9784.000046,9788.0,1.950740,2.950698,1.950703,11422.4,11422.000004,11422.1
Q,1.771078,1.771078,1.771078,9784.0,9784.000000,9784.0,1.950698,1.950698,1.950698,11422.0,11422.000000,11422.0
